In [1]:
!pip install ucimlrepo

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
y['income'] = y['income'].str.replace(r'\.$', '', regex=True)

  
# metadata 
print(adult.metadata) 
  
# variable information 
print(adult.variables) 


{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Tue Sep 24 2024', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': "Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the fol

C:\Users\Usuario\AppData\Local\Temp\ipykernel_1020\1752497249.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['income'] = y['income'].str.replace(r'\.$', '', regex=True)


In [4]:
X = X.dropna()
y = y.loc[X.index]

In [5]:
from sklearn.preprocessing import LabelEncoder

X_encoded = pd.get_dummies(X, columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])

le = LabelEncoder()
y['income'] = le.fit_transform(y['income'])

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


n_shadow = 5
n = len(X_encoded)//(n_shadow+1)

target_X = X_encoded.iloc[:n]
target_y = y.iloc[:n]

X_train, X_test, y_train, y_test = train_test_split(target_X, target_y, test_size=0.5)

target_in =X_train.index
target_out = X_test.index

target_model = RandomForestClassifier(n_estimators=100)
target_model.fit(X_train, y_train)

# 5. Hacer predicciones en el conjunto de prueba
y_pred = target_model.predict(X_test)

# 6. Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print("\nPrecisión del modelo Random Forest:", accuracy)

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

c:\Users\Usuario\anaconda3\envs\image_detection\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Precisión del modelo Random Forest: 0.8530745967741935

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.89      0.92      0.91      3057
           1       0.71      0.61      0.66       911

    accuracy                           0.85      3968
   macro avg       0.80      0.77      0.78      3968
weighted avg       0.85      0.85      0.85      3968



In [7]:
result_df = pd.DataFrame(columns=['prediction', 'in_or_out'])


for i in range(1, n_shadow+1):
    X_shadow = X_encoded.iloc[(n*i):(n*(i+1))]
    y_shadow = y.iloc[(n*i):(n*(i+1))]
    X_train, X_test, y_train, y_test = train_test_split(target_X, target_y, test_size=0.5)
    
    shadow_out = list(X_test.index)
    
    model_rf = RandomForestClassifier(n_estimators=100)
    model_rf.fit(X_train, y_train)

    preds = model_rf.predict_proba(X_shadow)

    temp_df = pd.DataFrame({
        'prediction': preds.tolist(),
        'in_or_out': [1] * len(X_shadow)
    })
    temp_df.loc[temp_df.index.isin(shadow_out), 'in_or_out'] = 0

    result_df = pd.concat([result_df, temp_df], ignore_index=True)
    

result_df.dropna()

c:\Users\Usuario\anaconda3\envs\image_detection\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Usuario\anaconda3\envs\image_detection\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Usuario\anaconda3\envs\image_detection\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\Usuario\anaconda3\envs\image_detection\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector 

,prediction,in_or_out
0,"[0.7, 0.3]",0
1,"[1.0, 0.0]",1
2,"[1.0, 0.0]",0
3,"[0.91, 0.09]",0
4,"[0.99, 0.01]",0
...,...,...
39675,"[0.52, 0.48]",0
39676,"[0.58, 0.42]",0
39677,"[0.66, 0.34]",1
39678,"[1.0, 0.0]",1


In [8]:
from Dataset import Standard_Dataset
import torch
from models import SimpleNN
from train import train_model
from torch.utils.data import random_split, DataLoader

num_epochs = 100
bsize = 16

loader = {}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = Standard_Dataset(result_df['prediction'], result_df['in_or_out'])

train_dataset, test_dataset = random_split(data, [0.8, 0.2])


model = SimpleNN()

model.to(device)

loss_func = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


loader['train'] = DataLoader(train_dataset, batch_size=bsize, shuffle=True)
loader['test'] = DataLoader(test_dataset, batch_size=bsize, shuffle=False)

train_model(model, loss_func, device, loader, optimizer, num_epochs)


Epoch 1/100
----------
tensor([[0.6900, 0.3100],
        [0.7000, 0.3000],
        [0.7900, 0.2100],
        [0.2000, 0.8000],
        [0.9000, 0.1000],
        [0.9600, 0.0400],
        [0.5400, 0.4600],
        [1.0000, 0.0000],
        [0.7200, 0.2800],
        [0.0300, 0.9700],
        [0.0600, 0.9400],
        [0.9900, 0.0100],
        [0.6200, 0.3800],
        [0.8000, 0.2000],
        [1.0000, 0.0000],
        [0.4100, 0.5900]], device='cuda:0') tensor([0.4711, 0.4711, 0.4707, 0.4821, 0.4698, 0.4692, 0.4733, 0.4691, 0.4710,
        0.4879, 0.4869, 0.4691, 0.4720, 0.4706, 0.4691, 0.4760],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0.])
tensor([[0.9000, 0.1000],
        [0.7800, 0.2200],
        [0.5000, 0.5000],
        [0.0600, 0.9400],
        [0.8400, 0.1600],
        [0.1400, 0.8600],
        [0.4400, 0.5600],
        [0.9700, 0.0300],
        [0.8900, 0.1100],
        [0.4700, 0.5300],
        [0.99

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
